In [8]:
import os
import h5py
import json
import mmap
import pickle
import numpy as np
from tqdm.auto import tqdm
from numba.typed import Dict, List
from numba import types
from transformers import (
    BertTokenizer
)

In [3]:
with open("/datacosmos/local/User/baoht/onesparse2/hybridsearch/bm25/index/marco/index_dist.json") as f:
    bm25_en_dist = json.load(f)

In [12]:
bm25_en_dist = dict(sorted(bm25_en_dist.items(), key=lambda item: int(item[0])))

In [15]:
dict_temp = {}
for k, v in bm25_en_dist.items():
    dict_temp[int(k)] = v
dict_temp = dict(sorted(dict_temp.items(), key=lambda item: item[0]))

In [3]:
total = 0
filename = "/datacosmos/local/User/baoht/onesparse2/marcov2/data/collection_train"
file_lists = [os.path.join(filename, f'part-{i}') for i in range(20)]

def count_lines(filename):
    cnt = 0 
    with open(filename, 'rb') as f:
        for line in f:
            cnt += 1
    return cnt
            
for file in file_lists:
    lens = count_lines(file)
    total += lens
    print("File {} has {} lines".format(file, lens))
print(total)

KeyboardInterrupt: 

In [ ]:
ids1 = [101, 102, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 102, 24432, 119, 10241, 131, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 131, 13374, 111, 43163, 10107, 13374, 111, 43163, 10107, 1284, 38632, 111, 34969, 1284, 29268, 1284, 10305, 26494, 1284, 10305, 26494, 12985, 10107, 111, 41851, 1284, 10305, 26494, 12985, 10107, 14924, 10155, 40504, 37822, 10301, 119, 11312, 11530, 112, 162, 16332, 10704, 10362, 11526, 10372, 38863, 11229, 10346, 11677, 10104, 16913, 119, 16653, 10323, 16117, 10114, 65068, 10104, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 17514, 131, 87200, 10310, 11601, 39332, 170, 10200, 42942, 10390, 22467, 14924, 10155, 40504, 102]
ids2 = [101, 102, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 102, 24432, 119, 10241, 131, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 131, 13374, 111, 43163, 10107, 13374, 111, 43163, 10107, 1284, 38632, 111, 34969, 1284, 29268, 1284, 10305, 26494, 1284, 10305, 26494, 12985, 10107, 111, 41851, 1284, 10305, 26494, 12985, 10107, 14924, 10155, 40504, 37822, 10301, 119, 11312, 11530, 112, 162, 16332, 10704, 10362, 11526, 10372, 38863, 11229, 10346, 11677, 10104, 16913, 119, 16653, 10323, 16117, 10114, 65068, 10104, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 17514, 131, 87200, 10310, 11601, 39332, 170, 10200, 42942, 10390, 22467, 14924, 10155, 40504, 102]

ids0 = [101, 102, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 102, 24432, 119, 10241, 131, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 131, 13374, 111, 43163, 10107, 13374, 111, 43163, 10107, 1284, 38632, 111, 34969, 1284, 29268, 1284, 10305, 26494, 1284, 10305, 26494, 12985, 10107, 111, 41851, 1284, 10305, 26494, 12985, 10107, 14924, 10155, 40504, 37822, 10301, 119, 11312, 11530, 112, 162, 16332, 10704, 10362, 11526, 10372, 38863, 11229, 10346, 11677, 10104, 16913, 119, 16653, 10323, 16117, 10114, 65068, 10104, 87200, 10310, 25265, 11325, 35166, 166, 35176, 11240, 36379, 12972, 11134, 12792, 27489, 113, 11418, 114, 17514, 131, 87200, 10310, 11601, 39332, 170, 10200, 42942, 10390, 22467, 14924, 10155, 40504, 102]

In [11]:
missing_ids_test = ['190561195', '194856434', '156208643', '177680157', '139778931', '160503555', '192710297', '190561205', '158356640', '121851255', '209858435', '182714531', '167689867', '161245367', '192710291', '159099779', '201301760', '197005280', '119399497', '186266065', '155714806', '184760193', '145477512', '209893214', '156208650', '136889049', '121851228', '206365057', '180566770', '169093126', '152658437', '192710292', '208488834', '159099771', '138264441', '132589476', '139035246', '136889046', '171238219', '121851230', '147643250', '158356633', '203450528', '209299664', '123999614', '122594948', '160503558', '194856418', '149769933', '195679225', '115410863', '205597618', '180566765', '119704180', '181971854', '151023851', '118301377', '179825323', '202044571', '201301766', '204194092', '143170476', '147621073', '129038745', '186266070', '117558157', '141204990', '134740674', '119704163', '154063459', '199896773', '201301749', '203450529', '157988852', '177680168', '201301751', '206339778', '114006801', '115410876', '115410871', '123999612', '177680172', '201421201', '120960464', '123999622', '202044572', '138667179', '184862678', '206339789', '199035537', '195497567', '194856422', '171981643', '204194097', '132589499', '124743594', '204194087', '182714523', '126891365', '117558162', '138917651', '160503543', '133334523', '163392782', '187008307', '203450532', '169835169', '188411244', '141823579', '190561212', '164916411', '120447421', '202044564', '194856417', '119704161', '164796112', '126148892', '117190352', '113263985', '152658423', '115410877', '134622813', '145477515', '121533291', '187008303', '180566769', '139035247', '166947370', '154807221', '131184131', '201421203', '110312388', '144073522', '195600054', '162650180', '166947369', '136889036', '122594946', '156047841', '153945270', '110894783', '169835180', '203450513', '164088087', '149769925', '165948642', '184117979', '125987743', '180566761', '208488835', '151915587', '149769949', '201301768', '151915581', '165541631', '162650173', '132589478', '181971839', '128296384', '139035241', '134622814', '148363964', '182714527', '129038738', '139303490', '134740687', '134740671', '169835179', '111859612', '156951084', '141182269', '130441794', '118301385', '207745360', '171238218', '144073527', '187008306', '201421204', '186266080', '169835173', '181971843', '123999606', '115410862', '156208658', '132589495', '179825302', '179825296', '208194489', '203450526', '206101019', '156951080', '175533951', '156951082', '177618147', '197748536', '122594954', '117558180', '114817821', '189155285', '111117074', '177680176', '141925964', '156208661', '117558153', '175533967', '130441808', '136889035', '130441802', '207745361', '145477497', '180835036', '151547828', '179825307', '186266074', '128178828', '192592604', '164044524', '168462332', '177680174', '114659753', '182714530', '201301769', '145477509', '185475082', '176276925', '171981640', '144886913', '197005283', '119704165', '144073528', '158356646', '125246288', '173576209', '145477505', '171981649', '190400231', '151915569', '126148900', '183321551', '152658432', '191004148', '201183540', '173385576', '169093123', '194856421', '200685885', '201301765', '189155289', '207745343', '156951087', '154704767', '190561191', '197005301', '123999634', '112183505', '156208663', '203450520', '126148894', '198931362', '126891372', '173385582', '169093133', '141925971', '209775233', '201140921', '160503550', '120447406', '164796111', '111859613', '186266067', '144073531', '115410873', '145596993', '158476786', '151915593', '124743601', '206642615', '144073523', '154807210', '126891368', '206046813', '203450534', '181971849', '151915577', '124743596', '190400232', '136889032', '197005287', '139035250', '184862677', '203450511', '164796113', '180566771', '125359955', '130441790', '146529727', '123999608', '209893217', '141182275', '149769948', '181971846', '130208960', '164270447', '176276928', '188411247', '173385573', '171238201', '120447412', '163392786', '189807528', '193451595', '136889050', '171078232', '146217679', '203290191', '121049595', '136889040', '151915584', '182714519', '146217678', '207745362', '114659803', '160622609', '171238194', '145047159', '186714900', '113103502', '199153101', '149769938', '147621084', '160503552', '113263972', '163392777', '174128905', '169835166', '207745345', '131184144', '154063470', '192710286', '113263970', '199153106', '127212186', '187008299', '149769927', '177680180', '173385581', '141182277', '189155291', '123999626', '140431381', '154807213', '203450517', '137631020', '199896770', '147254257', '197005293', '152658425', '199896778', '175654090', '113263969', '164796120', '121690825', '160503557', '138874998', '123999625', '166786890', '159751566', '202044566', '134740686', '194856432', '149769944', '201140920', '143330742', '126891364', '136520538', '199153111', '171238198', '132471725', '122594955', '184117982', '188064452', '147621072', '189155278', '162650181', '209893199', '177680166', '132471723', '168932140', '121851253', '153837051', '138264442', '133334519', '118007924', '121851236', '197005278', '202044570', '169835165', '204506168', '162650178', '117240137', '130441799', '158356639', '184862688', '134740688', '148631515', '171238211', '141182273', '171238214', '123999631', '172860277', '129038748', '166947381', '151918720', '156208651', '119704172', '197005295', '118953629', '207745344', '174630708', '164796128', '158648980', '162865675', '161245365', '147621085', '199153104', '199153112', '149769941', '149769953', '171238215', '149769922', '141182268', '166947375', '204194099', '201301758', '132589479', '143330734', '141925965', '141371293', '1253662', '132589500', '117397784', '208386204', '180566758', '117166583', '178129371', '139035251', '158196293', '111117075', '119586837', '133334524', '169835160', '166947383', '190400228', '130441800', '201301764', '136294438', '190897056', '119704188', '156951079', '169093122', '149769942', '197748544', '134740682', '118301380', '132589484', '201415246', '195600049', '162650166', '128296376', '123999611', '199153110', '145255277', '169835171', '136889033', '133334520', '180566755', '160503551', '128296386', '127928066', '168932139', '184117989', '111117078', '154063462', '162650175', '188411232', '183057695', '154807219', '203450522', '186385439', '166947377', '190561213', '139778934', '182714528', '201421205', '152658424', '192710282', '179825310', '179507213', '176859028', '202044563', '121851241', '194856433', '190561194', '131817231', '187523674', '134422053', '193451596', '149769932', '205597622', '161879355', '197005297', '176356208', '190561204', '197748542', '202044569', '150409765', '169835176', '122916189', '205597608', '114006800', '128376892', '113384329', '126148905', '123999618', '122594945', '144073534', '176910619', '184117973', '173385575', '197748553', '176276933', '207745355', '164796121', '119704181', '156208654', '175533969', '164796129', '121733078', '123999609', '157988871', '180464545', '184862676', '179825304', '162650174', '126148903', '146217677', '126891366', '145477504', '207627021', '177680167', '188411245', '165541638', '171238212', '165541635', '139526187', '111859606', '150512033', '139035249', '153270015', '183366628', '149769952', '183842397', '156208637', '203450523', '162650176', '199153105', '137631017', '147621080', '142857405', '119704160', '197005299', '132589496', '117558171', '169093139', '159751281', '143330740', '128296380', '119704187', '192829702', '166829681', '198784978', '123999623', '188411248', '184862683', '206339775', '169835175', '149054730', '141021716', '132589481', '154807216', '208488840', '205597627', '111117082', '169093131', '169835177', '182714522', '111117077', '190561206', '176276926', '171238206', '189234238', '143450223', '176276927', '165541643', '123999610', '197005284', '145477503', '113263976', '160167934', '135484233', '209893213', '143212735', '199153109', '162650167', '111859611', '186266078', '192710289', '150512030', '116154268', '121851227', '179111803', '164206563', '169093128', '162650184', '169093124', '199153096', '177680169', '130441791', '201301753', '199153103', '126148896', '184117980', '135484218', '114006795', '150774086', '151915576', '170102814', '146217681', '185232803', '156951075', '189155283', '152658430', '192549794', '113263980', '131182423', '148363962', '126788735', '207745352', '159099776', '171238210', '210535239', '126747497', '149769931', '139035242', '130441801', '199153099', '143212734', '130441803', '122594961', '131082091', '164635637', '172634121', '184117984', '138989345', '167689874', '137631016', '169835167', '160503567', '158997930', '121851232', '165541640', '171981646', '143098747', '127614145', '117558166', '139778940', '206339776', '184862687', '164428409', '162162668', '145477498', '160503547', '136889045', '149769957', '111398674', '120526477', '171981637', '173385586', '197125210', '162650168', '111875103', '144886912', '210637593', '113384331', '197005279', '197748543', '186266069', '193440601', '151915588', '121851225', '156951081', '179825315', '208386203', '121883884', '149769921', '206339777', '166947385', '151881817', '199896772', '177680155', '197729642', '111117083', '179825301', '151915590', '182714529', '172293264', '178422512', '111117080', '190400229', '201777689', '119704168', '201140922', '194856427', '175533962', '207745349', '144757187', '194262488', '164026066', '128564737', '128296387', '123999605', '194856424', '179825311', '188411243', '159099780', '136889034', '119704184', '164796123', '145477494', '121236637', '156951077', '186266068', '165620958', '202044556', '175373274', '134740667', '121851240', '167689871', '194856439', '151915571', '156208665', '139035244', '118301378', '111859608', '207745353', '167689861', '135484222', '193451588', '163392781', '123839170', '169093135', '179825313', '162281796', '150512025', '177064971', '200827923', '154063469', '121851252', '174128902', '147810176', '186266071', '188411246', '121262472', '122594947', '145477496', '115410864', '150512024', '141182271', '161747481', '169835158', '179825317', '194856438', '179825295', '154063463', '160503548', '199896780', '110364285', '165948643', '132607134', '182714526', '151915572', '160503571', '143330731', '197005275', '154063465', '124660035', '119089494', '67260307', '169093132', '164270448', '193975134', '138243368', '187008291', '181220209', '147254293', '118301370', '118301374', '179825314', '160503564', '163471908', '136889029', '117558167', '181971845', '123409657', '128296366', '119704175', '192036646', '206952739', '170117149', '126148890', '177609308', '149769936', '195600048', '193451589', '166947382', '149769947', '136728227', '117558173', '199896777', '169835159']

In [12]:
with open("data/missing_ids_test.pkl", "wb") as f:
    pickle.dump(missing_ids_test, f)

In [13]:
passage_path = "/datacosmos/local/User/baoht/onesparse2/marcov2/data/training_data"
id2id = json.load(open(os.path.join(passage_path,'id2id_test.json')))

In [18]:
missing_ids_test_set = set([int(x) for x in missing_ids_test])
with open("data/collection_test/part-0", encoding="utf-8") as f:
    for line in tqdm(f):
        lines = line.split('\t')
        url, language, doc_id, title, body = lines
        if int(doc_id)  in missing_ids_test_set:
            print(line)
            print(len(line.split('\t')))
            break

58273it [00:01, 35136.01it/s]

https://listaspam.com/busca.php?telefono=25095000	es	190561195	¿Quién me llama del teléfono 25095000? | Denuncias - ListaSpam	 

5


In [19]:
missing_ids_train = ['268815', '611465', '717722', '1126183', '1253656', '1253659', '1253662', '1253663', '1253664', '1253665', '1253666', '1253669', '1253674', '1253677', '1253680', '1253682', '1253684', '1253686', '2061189', '2061190', '2061197', '2061198', '2061200', '2061201', '2061202', '2774217', '2851974', '3247012', '3419025', '3593191', '3593195', '3593196', '3593198', '3593199', '3593205', '3593207', '3724597', '3724598', '4401233', '4401236', '4401238', '4401240', '4401246', '4704641', '5437843', '5587918', '5689254', '5759970', '5934810', '5934818', '5934824', '5934827', '5934828', '5934829', '5934830', '5934836', '6742899', '6742900', '6742902', '6742905', '6742908', '6742910', '7454451', '7454498', '8100765', '8100768', '8100769', '8276085', '8276088', '8276089', '8276090', '8276091', '8276092', '8276094', '8970544', '8970545', '9082141', '9082142', '9082145', '9082146', '9082154', '9484031', '9629697', '9751434', '9943174', '10438758', '10613307', '10613308', '10613309', '10613312', '10613313', '10613314', '10613316', '10613317', '10613318', '10613320', '10613321', '10613323', '10613324', '10613328', '11145569', '11421544', '11421549', '11777312', '11968640', '12553295', '12827160', '12953852', '12953855', '12953856', '12953858', '12953861', '12953866', '12953867', '12953875', '12957352', '13085695', '13761336', '13761342', '13761346', '13761348', '13761349', '13847813', '14429480', '14472908', '14514152', '15112771', '15119113', '15166753', '15200686', '15294112', '15294114', '15294116', '15294119', '15294120', '15294122', '15294124', '15294125', '15318792', '16101266', '16101267', '16101270', '16101273', '16813797', '16962958', '17459367', '17633684', '17633685', '17633686', '17633690', '17633691', '17633695', '17633696', '17633698', '17633699', '17633700', '17633701', '17633704', '17633705', '17764753', '17764754', '17840714', '18298428', '18329870', '18441609', '18441610', '18441614', '18441617', '18441623', '18441624', '18528502', '19573677', '19974621', '19974622', '19974623', '19974624', '19974625', '19974628', '19974630', '19974631', '19974634', '19974635', '19974636', '19974637', '19974638', '19974640', '19974642', '19974645', '19974648', '20782696', '20782702', '20782706', '20782710', '20782713', '21475927', '22022624', '22140190', '22315387', '22315388', '22315390', '22315396', '22315397', '22315399', '22315401', '22315403', '22315404', '22315405', '22446679', '23123106', '23123108', '23123111', '23210179', '23985003', '24655116', '24655118', '24655120', '24655122', '24655124', '24655125', '24655127', '24655129', '24655134', '24655135', '24655137', '24655138', '24861441', '25462431', '25462432', '25462433', '25462435', '25462437', '25462439', '25840640', '26173271', '26993604', '26993613', '26993617', '26993618', '26993621', '26993624', '26993625', '26993628', '26993631', '26993632', '26993633', '27124949', '27172751', '27801558', '27801559', '27801561', '27801563', '27801565', '27801569', '28494046', '29159224', '29206545', '29333452', '29333456', '29333457', '29333458', '29333461', '29333468', '29333474', '29333475', '29479045', '30030298', '30030299', '30142614', '30142616', '30142618', '30142619', '30142622', '30856356', '30856369', '31377693', '31502225', '31549578', '31677221', '31677223', '31677224', '31677227', '31677229', '31677232', '31677233', '31677234', '31677235', '31677236', '32266304', '32485002', '32485003', '32485006', '32485008', '32485011', '32824769', '33142753', '33617426', '33671538', '33843893', '34018211', '34018213', '34018214', '34018215', '34018218', '34018219', '34018220', '34018225', '34018226', '34018229', '34018230', '34277750', '34690015', '35838917', '36356722', '36356723', '36356724', '36356725', '36356729', '36356730', '36356733', '36356735', '36356736', '36356738', '36489535', '37165749', '37165758', '37165759', '37165761', '37165762', '37165765', '37835200', '37878620', '38027680', '38297016', '38697949', '38697950', '38697951', '38697955', '38697961', '38697967', '38697968', '38697969', '38697972', '38697974', '38829712', '38829713', '39312799', '39506636', '39506638', '39506645', '39506646', '40160199', '40394072', '40794912', '40860892', '41035366', '41035367', '41035369', '41035371', '41035374', '41035378', '41035380', '41035382', '41035384', '41035385', '41035386', '41035387', '41035388', '41035391', '41035392', '41242796', '41842468', '41842473', '41842475', '41842476', '41842478', '41842481', '41842484', '41842486', '41928851', '42389803', '42389804', '42731681', '43199458', '43246809', '43373916', '43373917', '43373924', '43373926', '43373927', '43373928', '43373930', '43373931', '43373932', '43373935', '43581247', '43861764', '44181046', '44181047', '44181049', '44181051', '44181058', '44181062', '44874172', '45172973', '45538232', '45585928', '45707114', '45713390', '45713391', '45713395', '45713396', '45713402', '45713403', '45713408', '45713409', '45713410', '45713413', '45713414', '46410056', '46522413', '46522417', '46522418', '46522421', '46522423', '47233576', '47812010', '48051979', '48051980', '48051982', '48051983', '48051985', '48051986', '48051988', '48051992', '48051994', '48052002', '48183509', '48858484', '48858485', '48858487', '48858490', '48858495', '48858496', '48858497', '48858500', '50216328', '50391338', '50391344', '50391347', '50391359', '50522530', '50614158', '51198732', '51198740', '51198741', '51198744', '51198747', '51891973', '51911275', '52556388', '52731105', '52731108', '52731109', '52731110', '52731114', '52731115', '52731118', '52731120', '52731122', '52731123', '52731125', '52731130', '52731133', '52731137', '52731138', '52862854', '53539262', '53539264', '53539268', '53539270', '53539271', '53539274', '54895118', '55069808', '55069810', '55069811', '55069813', '55069815', '55069817', '55069818', '55069819', '55069820', '55069822', '55069826', '55069827', '55069831', '55069832', '55156493', '55877385', '55877387', '55877390', '55877391', '55877394', '55877396', '57409427', '57409429', '57409437', '57409438', '57409439', '57409441', '57409442', '57409443', '57409451', '57434546', '57540997', '58218161', '58218171', '59278650', '59624239', '59751842', '59751843', '59751845', '59751846', '59751850', '59751851', '59751852', '59751853', '59751858', '59751859', '59751860', '59751861', '59751865', '59751867', '59751868', '59825138', '59920205', '60344172', '60447015', '60447016', '60558363', '60558365', '60558371', '61105823', '61251261', '61271403', '61518674', '61964235', '62091557', '62091560', '62091561', '62091562', '62091563', '62091565', '62091566', '62091567', '62091569', '62091571', '62276060', '62309191', '62900119', '62900121', '62900124', '63448181', '63571275', '63975238', '64258752', '64306403', '64433453', '64433454', '64433457', '64433458', '64433459', '64433460', '64433462', '64433463', '64433465', '64433467', '64433468', '64820198', '64913148', '64979354', '65175833', '65241213', '65241216', '66305219', '66772848', '66772850', '66772851', '66772852', '66772853', '66772855', '66772856', '66772859', '66772866', '66772868', '66772869', '66772873', '67260307', '67469950', '67581214', '67581221', '67581222', '67581223', '67581225', '68469414', '68938081', '69112363', '69112369', '69112372', '69112377', '69112378', '69112381', '69112384', '69112385', '69112388', '69808576', '69919628', '69919631', '69919634', '69919638', '70741520', '71451784', '71451786', '71451789', '71451790', '71451791', '71451793', '71451795', '71451799', '71451803', '71528145', '71582907', '72257546', '72257556', '72257565', '72257566', '73141816', '73272158', '73488843', '73542410', '73787031', '73787032', '73787038', '73787042', '73787043', '73787047', '73787048', '74595703', '74595705', '74595707', '75135705', '75289322', '76128540', '76128545', '76128549', '76128550', '76128552', '76128556', '76128557', '76128558', '76450625', '76937131', '76937135', '76937136', '76937138', '77293105', '78471290', '78471292', '78471295', '78471296', '78471298', '78471299', '78471301', '78471302', '78471303', '78471309', '78471310', '78471315', '78471316', '78471321', '79003449', '79167630', '79279056', '79279057', '80142074', '80239945', '80410684', '80811895', '80811898', '80811900', '80811904', '80811905', '80811906', '80811908', '80811909', '80811912', '80811913', '80811914', '80811921', '80811922', '80811923', '81091880', '81619013', '81619014', '81619015', '81619017', '81619018', '81619023', '82273482', '82507721', '82660874', '82974995', '83022184', '83149348', '83149349', '83149350', '83149353', '83149357', '83149358', '83149359', '83149360', '83149361', '83149362', '83149368', '83149370', '83149371', '83168381', '83281299', '83956042', '83956044', '83956046', '83956048', '83956050', '83956051', '83956054', '83956056', '83956057', '83956058', '83956061', '83956062', '84042082', '84647408', '85018828', '85486263', '85486264', '85486265', '85486268', '85486269', '85486271', '85486273', '85486275', '85486277', '85486278', '86293522', '86293525', '86293526', '86293528', '86293531', '86572523', '87648020', '87822639', '87822640', '87822641', '87822642', '87822644', '87822647', '87822650', '87822651', '87822652', '87822653', '87822654', '87822658', '87822659', '87822660', '87822661', '87822662', '87822663', '88629045', '88630978', '88630980', '88630983', '88630984', '89762504', '89979090', '90162900', '90162905', '90162906', '90162908', '90162909', '90162910', '90162913', '90162914', '90162915', '90162916', '90162917', '90162919', '90162922', '90162923', '90162924', '90162926', '90162933', '90294210', '90858529', '90858531', '90970126', '90970128', '90970129', '90970133', '90970135', '91518481', '91832530', '92502411', '92502412', '92502413', '92502415', '92502416', '92502420', '92502421', '92502423', '92502425', '92502428', '92502429', '92502430', '92502431', '92502432', '92502435', '92633892', '93308667', '93308671', '93308674', '93308675', '93967284', '94440117', '94440119', '94665989', '94840874', '94840875', '94840877', '94840878', '94840879', '94840880', '94840882', '94840883', '94840886', '94840887', '94840889', '94840892', '95049386', '95649310', '95649311', '95649316', '95649317', '95649319', '95649321', '95649326', '95649327', '95649332', '95649333', '96343007', '97006284', '97053865', '97181353', '97181355', '97181357', '97181360', '97181365', '97181366', '97181367', '97181368', '97489304', '97988961', '97988963', '97988969', '97988973', '97988975', '97988977', '99521516', '99521519', '99521520', '99521521', '99521523', '99521524', '99521525', '99521526', '99521527', '99641052', '100330958', '100330960', '100330961', '100330962', '100330963', '100330965', '100330967', '100330969', '100617735', '101045247', '101623474', '101689233', '101689235', '101864454', '101864458', '101864461', '101864466', '101864467', '101864472', '101864475', '101864476', '101864480', '101864481', '101995773', '102559438', '102671371', '102671372', '102671378', '102671380', '103318438', '103561720', '104029051', '104029052', '104203611', '104203612', '104203615', '104203616', '104203618', '104203619', '104203622', '104203623', '104203624', '104203625', '104203628', '104203631', '104203632', '104203634', '104203636', '105012076', '105012077', '105705450', '105705451', '105725539', '105874478', '105972717', '106370265', '106545411', '106545412', '106545414', '106545419', '106545423', '106545424', '106545426', '106545427', '107351004', '107351008', '107351013', '107351016', '107351019', '107402493', '108310643', '108482347', '108482411', '108883454', '108883457', '108883458', '108883460', '108883462', '108883463', '108883465', '108883469', '108883470', '108883472', '108883474', '109579275', '109690341', '109690344', '109690345', '109690350', '109690351']

In [22]:
print(len(missing_ids_train))
with open("data/missing_ids_train.pkl", "wb") as f:
    pickle.dump(missing_ids_train, f)

976
